# Observation Planning Part 3


1. Where is my target?
2. When can I observe my target?
3. How do I know when I've found my target (make a finder chart)?
4. How long do I need to observe?
5. __How will I calibrate my data?__


# Index

* [Q1](#Q1): What factors affect our ability to collect light?
* [Calibration: Photometric Standards](#Calibration:-Photometric-Standards)
  * [Closest Calibrator](#Find-closest-calibrator)
  * [Finder Chart for calibrator](#Finder-Chart-for-Calibrator)
  * [Exposure Time for calibrator](#Exposure-time-for-calibrator)

## How will I calibrate my data?

To make a photometric measurement, we need to know how efficient our instrument really is.  In theory, every photon collected in our telescope goes to the CCD and is converted to electrons, but in practice several factors prevent this.  

# Q1

What are the factors that limit our ability to collect the light?  i.e., what prevents us from counting every photon?  Consider instrumental and atmospheric effects.

(student answer)

## Calibration: Photometric Standards

To calibrate our image, we can use reference stars with known brightness to infer how much light is lost on the way to our image.

**Landolt photometric standard stars** are the best standards to use because they've been selected to be non-variable and have been carefully calibrated.  

However, the AAVSO (American Association of Variable Star Observers) has provided some really nice tools for obtaining standard star locations:

https://www.aavso.org/apps/vsd/stdfields

We can also retrieve catalogs from Vizier, the Centre de Données astronomiques de Strasbourg services for catalogs.

For simplicity, we'll adopt this latter approach.

In [ ]:
from astroquery.vizier import Vizier

You can use the `Vizier.get_catalogs` method in astroquery to grab the data from a catalog whose precise name you already know

In [ ]:
Vizier.ROW_LIMIT = 10000
landolt_catalog = Vizier.get_catalogs('J/AJ/146/131/standards')

Vizier actually returns a list of tables:

In [ ]:
landolt_catalog

But we want to look at only the first of these:

In [ ]:
landolt_tbl = landolt_catalog[0]
landolt_tbl.show_in_notebook(display_length=5) # you can change the display_length to show more or fewer rows

The table above contains two columns that specify the RA and Dec coordinates, `RAJ2000` and `DEJ2000`.

We'd like to be able to plot these and use them in some calculations below, so we'll turn them into `astropy.coordinates` objects.   Note that we need to do this because most software tools don't know how to read sexagesimal labels as numbers.  Anyway, the conversion process is easy:

In [ ]:
from astropy import coordinates, units as u

In [ ]:
landolt_coords = coordinates.SkyCoord(landolt_tbl['RAJ2000'], landolt_tbl['DEJ2000'], unit=(u.h, u.deg))

We can visualize the _sky coverage_ of these standard stars in RA and Dec:

In [ ]:
import pylab as plt
plt.style.use('dark_background') # if you're using a light background, you should turn this off

In [ ]:
_=plt.plot(landolt_coords.ra, landolt_coords.dec, 'o')
_=plt.xlabel("RA")
_=plt.ylabel("Dec")
_=plt.title("Sky locations of Landolt standard stars (North)")

What if we want to figure out where these are in altitute and azimuth (alt/az) relative to us, the observer?  The Sky Plot feature from astroplan is good for that!  Of course, we need to specify the observatory first:

In [ ]:
from astroplan import Observer
from astropy import units as u # shortcut
CTO = Observer(location=coordinates.EarthLocation(lat=29.643018, lon=-82.349004*u.deg, height=31*u.m),
               timezone='EST',
               name='University of Florida Campus Teaching Observatory',
              )

In [ ]:
from astroplan.plots import plot_sky
from astropy.time import Time

In [ ]:
_=plot_sky(target=landolt_coords, observer=CTO, time=Time('2021-04-11 18:00:00'))
_=plot_sky(target=landolt_coords, observer=CTO, time=Time('2021-04-11 20:00:00'))
_=plot_sky(target=landolt_coords, observer=CTO, time=Time('2021-04-11 22:00:00'))
_=plt.title("Sky Plot relative to CTO at 1800, 2000, and 2200 on April 11")

There are a lot of targets that go straight overhead!  There aren't so many in the South, but that's because we picked a standard star catalog that is meant for the north; if we wanted stars further south, we could use ["II/183A/table2"](http://vizier.u-strasbg.fr/viz-bin/VizieR-3?-source=II/183A/table2) instead.

To find these table IDs, you can just search on Vizier for the name "Landolt standards".

Let's say we want to observe P Cygni.  How do we find the closest standard(s)?

First, we can simply overplot it on our RA/Dec plot

In [ ]:
pcyg_coord = coordinates.SkyCoord.from_name('P Cygni')

In [ ]:
_=plt.plot(landolt_coords.ra, landolt_coords.dec, 'o')
_=plt.plot(pcyg_coord.ra, pcyg_coord.dec, 's')
_=plt.xlabel("RA")
_=plt.ylabel("Dec")
_=plt.title("Sky locations of Landolt standard stars (North) + P Cyg")

## Find closest calibrator

[Index](#Index)

How can we figure out which calibrator source from the catalog is the closest one?

We can calculate the distance between P Cygni and each of the stars in the Landolt catalog.  That distance is the _angular separation_ on the sphere:
$$\theta = \cos^{-1} \left[ \sin(\delta_1) \sin(\delta_2) + \cos(\delta_1) \cos(\delta_2) \cos(\alpha_1 - \alpha_2) \right]$$

In practice, you don't want to do this yourself, as there can be numerical issues when calculating these values near the poles (see the article on [Great Circle distances](https://en.wikipedia.org/wiki/Great-circle_distance)).  Thankfully, astropy's coordinates provide a `separation` tool to calculate this for us.



In [ ]:
distances_from_pcyg_to_standards = pcyg_coord.separation(landolt_coords)

We can then find which of these is closest by taking the minimum:

In [ ]:
np.min(distances_from_pcyg_to_standards)

The closest standard star is about 2 degrees away.  Which star is it, though?  We can use `np.argmin` to obtain the index corresponding to that minimum value.

In [ ]:
index = np.argmin(distances_from_pcyg_to_standards)
index, landolt_coords[index]

So now we have its location.  Can we find out more about the star, like its name and brightness?

Since `landolt_coords` has the same length and order as the `landolt_table` above, yes!  We can use the same index:

In [ ]:
landolt_tbl[index]

Great!  We've found our standard star, and we know it has a visual magnitude $V_{mag} = 12.5$!

In [ ]:
print("V Magnitude of standard star: ",landolt_tbl[index]['__Vmag_'])

We can also determine its magnitude in the B and R bands using the colors in the table.  Note that the titles of the columns tell you what they contain: except for the V-band, the columns show _colors_, i.e., delta-magnitudes.  The `B-V` column is the `B-V` color.  If you want to obtain the B color, you just do `B-V + V = B`.  

In [ ]:
print("B Magnitude of standard star: ",landolt_tbl[index]['__Vmag_'] + landolt_tbl[index]['__B-V_'])
print("R Magnitude of standard star: ",landolt_tbl[index]['__Vmag_'] - landolt_tbl[index]['__V-R_'])

Now that we've selected this star, we need to go back through and do the same planning exercises for it as for the targets:

1. Where is my calibrator?  (we answered this a few cells ago)
2. When can I observe my calibrator? (the same time as my target!)
3. How do I know when I've found my calibrator? (make a finder chart)
4. How long do I need to observe?

## Finder Chart for Calibrator
[Index](#Index)

Since we've done (1) and (2), let's do (3) and (4):

Recall how you made finder charts from Observation Planning Exercise 1:

In [ ]:
from astroplan.plots import plot_finder_image
ax, hdu = plot_finder_image(landolt_coords[index], survey='DSS', fov_radius=15*u.arcmin, grid=False, reticle=True)

If you've been on an observing run, you know that these finder charts are hard to use, and sometimes it's better to show them more saturated.  You can do that by specifying `style_kwargs`:

In [ ]:
ax, hdu = plot_finder_image(landolt_coords[index],
                            survey='DSS',
                            fov_radius=15*u.arcmin, grid=False, reticle=True,
                            style_kwargs={'vmin':18000, 'vmax':18100})

You can also change the colorscale if you want something that looks a little more like the night sky:

In [ ]:
ax, hdu = plot_finder_image(landolt_coords[index], survey='DSS',
                            fov_radius=15*u.arcmin, grid=False,
                            reticle=True, style_kwargs={'vmin':18000, 'vmax':18100, 'cmap':'gray'})

## Exposure time for calibrator

[Index](#Index)

How long do we need to observe?  We use the same technique as Observation Planning Exercise 2.

The zero points are:
* [3600 Jy for V-band](http://svo2.cab.inta-csic.es/theory/fps/index.php?id=Generic/Bessell.V&&mode=browse&gname=Generic&gname2=Bessell#filter)
* [4000 Jy for B-band](http://svo2.cab.inta-csic.es/theory/fps/index.php?id=Generic/Bessell.B&&mode=browse&gname=Generic&gname2=Bessell#filter)
* [2400 Jy for I-band](http://svo2.cab.inta-csic.es/theory/fps/index.php?id=Generic/Bessell.I&&mode=browse&gname=Generic&gname2=Bessell#filter)

In [ ]:
standard_star_vmag = landolt_tbl[index]['__Vmag_']
vmag_zeropoint = 3600*u.Jy
snu_standard = vmag_zeropoint * (10**(-standard_star_vmag/2.5))
standard_star_vmag, snu_standard

Using the telescope area and the filter properties, we now determine how much energy we receive from the star (which we calculated in the previous notebook):

* The V filter has a central wavelength of 5504 Angstroms
* The V filter has a width of "about" 1000 Angstroms (for [this filter](http://svo2.cab.inta-csic.es/theory/fps/index.php?id=Generic/Bessell.V&&mode=browse&gname=Generic&gname2=Bessell#filter), the width is 893 Angstroms, but we'll stick with the order-of-magnitude approximation for now)
* We are calculating the area of a 14-inch telescope

In [ ]:
v_filt_wav = 5504*u.AA
v_filt_freq = (v_filt_wav).to(u.Hz, u.spectral())
v_filter_width = 1000*u.AA
A_CTO = (np.pi*(14/2 * u.imperial.inch)**2).to(u.cm**2)
standard_ergs_per_s = snu_standard * A_CTO * v_filt_freq*(v_filter_width/v_filt_wav)
standard_ergs_per_s.to(u.erg/u.s)

In [ ]:
from astropy import constants

As before, we want to know the _number of photons_ we will receive per second, so we convert the above energy to a photon rate.

In [ ]:
standard_phot_per_s = (standard_ergs_per_s / (constants.h * v_filt_freq)).decompose()
standard_phot_per_s

Then, we want to account for the inefficiences in our telescope: the average filter efficiency, the CCD's quantum efficiency, and the loss from atmospheric absorption (noting, of course, that the atmospheric loss is calculated for zenith, so it could be worse than this!)

In [ ]:
filter_efficiency = 0.75
quantum_efficiency = 0.7
atmosphere_loss = 0.2
received_fraction = filter_efficiency * quantum_efficiency * (1-atmosphere_loss)
received_fraction

As before, we need to determine how much our signal will be spread out.  We use the same $\sigma=2"$ PSF and $0.5"$ pixel scale

In [ ]:
psf_area = 2 * np.pi * (2*u.arcsec)**2
psf_area

In [ ]:
pixel_scale = 0.5*u.arcsec/u.pixel
psf_area_pixels = psf_area * pixel_scale**-2
psf_area_pixels

In [ ]:
count_rate_per_pixel = standard_phot_per_s / psf_area_pixels * received_fraction
count_rate_per_pixel

We want our signal-to-noise ratio to be at least as good as our target, ideally better, since we will be comparing these measurements (taking their difference) and therefore their noise will add in quadrature again.  We can set a target SNR = 100 as before


Recall that the readnoise on the sum is the square root of the sum of the individual pixel read noise:

$$ \sigma_{RN,sum}^2 = \Sigma_i \sigma_{RN,i}^2 = N \sigma_{RN}^2$$
$$ \sigma_{RN,sum} = \sqrt{N}  \cdot \sigma_{RN}$$

In [ ]:
readnoise_per_pix = 10*u.adu/u.pix
readnoise_sum = psf_area_pixels**0.5 * readnoise_per_pix
readnoise_sum

Recall the equation (from Observation Planning Part 2) for the target signal as a function of SNR:

$$S = SNR^2 \pm \frac{\sqrt{SNR^4 + 4 \sigma_{RN}^2 SNR^2}}{2}$$

In [ ]:
SNR = 100
target_signal = SNR**2 + (SNR**4 + 4*readnoise_sum.value**2*SNR**2)**0.5 / 2
target_signal

In [ ]:
integration_time_including_readnoise = target_signal / standard_phot_per_s / received_fraction
integration_time_including_readnoise

# Exercise

[Index](#Index)

Repeat this exercise below for:

* (1) the B and R filters.  How long do you need to expose the standard star in each of those filters to get to SNR = 100?
* (2) A standard star near your target.  Use one of the targets we observed (e.g., M13 or M57).


Note that for most of this exercise, you don't need to re-calculate much of the above.  Think about which items you do need to re-calculate.  Answer these questions before you do the exercise.

Do you need to recalculate or re-enter:

* The star's magnitude? Yes
* The star's spectral flux density?  Flux density?
* The star's count rate?
* The PSF area?
* The total readnoise?
* The telescope area?
* The filter center?
* The filter width?
* The goal SNR?
* The target signal for the goal SNR?
* The goal integration time?
* The star location?


Answer each of the above with a "Yes" or "No".  The first is answered "Yes" for you - we're looking at a different filter for part (1) and a different target for part (2), so you definitely need to have a different value for the magnitude.

Then, in new cells below here, complete the exercise: